# Tensor network quantum control for expectation value minimisation

In this program, we take the optimised gate sequence from _vqe_from_expval_mpo.m_ and implement a state vector simulation to confirm they correctly minimising the expectation value.

For some reason, this is not working... even though we are correctly recovering the expectation value for the $n=4$ case. However, when we change the number of qubits, the expectation value starts failing too! wtf!

In [49]:
import numpy as np
import random
import itertools
import time
from matplotlib import pyplot as plt
from qutip import *

from pauli_string_functions_module_python import *

In [80]:
#crushed Ising model (cIm) - definition, spectrum and eigenstates
n = 8
X1, Xn = 'X' + 'I' * (n - 1), 'I' * (n - 1) + 'X'
cIm_terms = tuple(list(all_single_operator_strings(n, 'Z')) + [X1, Xn] + list(local_2body_strings(n, 'XX')))
H_cIm = 0 
for j in range(len(cIm_terms)):
    H_cIm += operator_string_from_Pauli_string(cIm_terms[j])

cIm_spectrum, cIm_states = H_cIm.eigenstates()
cIm_GS = cIm_states[0]
cIm_spectrum

array([-10.7792482 , -10.12762364,  -9.50278389,  -8.92177295,
        -8.4004393 ,  -8.19052456,  -7.95300355,  -7.60951362,
        -7.59167058,  -7.32629662,  -7.16412037,  -7.08817997,
        -6.98467387,  -6.64074422,  -6.46334022,  -6.33304931,
        -6.27941125,  -6.01590447,  -6.01403729,  -5.88232928,
        -5.85186104,  -5.81171565,  -5.6545715 ,  -5.43489353,
        -5.38919754,  -5.3642799 ,  -5.23070472,  -5.22702129,
        -5.07356057,  -5.00294693,  -4.91355988,  -4.8081866 ,
        -4.78326897,  -4.73757297,  -4.64601036,  -4.60586497,
        -4.57539672,  -4.55222691,  -4.421936  ,  -4.28685295,
        -4.26193531,  -4.15842922,  -4.15656204,  -4.1246767 ,
        -4.10479116,  -3.99438579,  -3.90060235,  -3.8394172 ,
        -3.79709625,  -3.67724095,  -3.63709556,  -3.63522838,
        -3.53172229,  -3.47808423,  -3.47305214,  -3.45316659,
        -3.36954604,  -3.31590799,  -3.29360564,  -3.2757626 ,
        -3.18779263,  -3.05608463,  -3.05053402,  -3.02

In [71]:
#importing and organising data
n = 4
bin_num = 10
ctrl_num = 8
x_optm = np.loadtxt('data/cIm_bin_num=10_n=4_test.csv', delimiter=',')
T_optm = x_optm[-1]
Dt = T_optm / bin_num
c_optm = np.reshape(x_optm[:-1], (bin_num, ctrl_num))

cX_temp, cY_temp = np.zeros([n, bin_num]), np.zeros([n, bin_num])
for i in range(n):
    cX_temp[i] = c_optm[:, 2 * i]
    cY_temp[i] = c_optm[:, 2 * i + 1]

#swapping sign and order of cX an cY
cX_binned, cY_binned = np.zeros([n, bin_num]), np.zeros([n, bin_num])
for i in range(n):
    cX_binned[i] = -np.flip(cX_temp[i])
    cY_binned[i] = -np.flip(cY_temp[i])
#row represents the time bin, collumn represents the qubit on which X or Y act

In [72]:
#state vector simulation parameters and time-dependent pulse definition
time_steps = 10000
times = np.linspace(0, T_optm, time_steps)

cX, cY = np.zeros([n, time_steps]), np.zeros([n, time_steps])
for i in range(n):
    count = 1
    for j in range(time_steps):
        if times[j] > count * Dt and count < bin_num: #second condition avoids floating error leading to count = bin_count + 1
            count += 1
        cX[i, j] = cX_binned[i, count - 1]
        cY[i, j] = cY_binned[i, count - 1]

X_ops, Y_ops = np.zeros(n, dtype = list), np.zeros(n, dtype = list)
H_control = []
for i in range(n):
    X_ops[i] = operator_string_from_Pauli_string(pauli_string_at_qubit_position(n, 'X', i))
    Y_ops[i] = operator_string_from_Pauli_string(pauli_string_at_qubit_position(n, 'Y', i))
    H_control.append([X_ops[i], cX[i]])
    H_control.append([Y_ops[i], cY[i]])

cZZ = np.ones(time_steps) # -ZZ as time reversed due to optimising for +ZZ (change this in MATLAB)
ZZ_ops = np.zeros(n, dtype = list)
H_native = []
for i in range(n - 1):
    ZZ_ops[i] = operator_string_from_Pauli_string(pauli_string_at_qubit_position(n, 'ZZ', i))
    H_native.append([ZZ_ops[i], cZZ])

H_time_dependent = H_control + H_native

In [73]:
#state vector simulation --> does it work!?!
psi0 = tensor([basis(2, 0)] * n)
result = mesolve(H_time_dependent, psi0, times)
psiT = result.states[-1]

In [74]:
expect(H_cIm, psiT), expect(H_cIm, cIm_GS)

(1.1240011819502451, -5.457414830239131)

In [78]:
H_control

[[Quantum object: dims = [[2, 2, 2, 2], [2, 2, 2, 2]], shape = (16, 16), type = oper, isherm = True
  Qobj data =
  [[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
   [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]],
  array([ 0.36859244,  0.36859244,  

In [75]:
fidelity(psiT, cIm_GS)

0.07882934612317968